In [ ]:
import time
start_time = list(time.localtime())[3:6] #returns the local time at the start of execution. Can be used to calculate the execution time manually

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import scipy.signal as sig
import control as con
from statsmodels.tsa.arima_model import ARIMA
import warnings
%pylab inline
warnings.filterwarnings('ignore') #ignores all the warnings that may arise during runtime

###### Reading all the data

In [ ]:
def read_data(control_output,task_vel):
    df_soll     = pd.read_csv(control_output, header = 0, names = ['time', 'x_soll'])
    df_soll     = df_soll.set_index('time')
    df_soll     = df_soll[~df_soll.index.duplicated(keep = 'first')] #gets rid of any duplicate index values if present
    df_ist      = pd.read_csv(task_vel, header = 0, names = ['time', 'x_ist'])
    df_ist      = df_ist.set_index('time')
    df_ist      = df_ist[~df_ist.index.duplicated(keep = 'first')]
    df_ist_soll = pd.concat([df_soll.x_soll, df_ist.x_ist], axis = 1).fillna(method = 'pad')
    df_ist_soll = df_ist_soll.fillna(0)
    return df_ist_soll


control_output_1 = 'data/2017-11-03_14-25-18_control_output.log'
task_velocity_1  = 'data/2017-11-03_14-25-19_task_vel.log'

control_output_2 = 'data/2017-11-03_14-26-43_control_output.log'
task_velocity_2  = 'data/2017-11-03_14-26-43_task_vel.log'

control_output_3 = 'data/2017-11-03_14-27-47_control_output.log'
task_velocity_3  = 'data/2017-11-03_14-27-47_task_vel.log'

control_output_4 = 'data/2017-11-03_14-30-43_control_output.log'
task_velocity_4  = 'data/2017-11-03_14-30-43_task_vel.log'

df_ist_soll_1 = read_data(control_output_1, task_velocity_1)
df_ist_soll_2 = read_data(control_output_2, task_velocity_2)
df_ist_soll_3 = read_data(control_output_3, task_velocity_3)
df_ist_soll_4 = read_data(control_output_4, task_velocity_4)
#df_ist_soll_1.plot(style = '-', drawstyle = "steps")

###### Removing all zeros & the negative trend and reformatting the data in accordance with a unit step response

In [ ]:
#function that strips zeros and multiplies the dataframe to 1
def strip_multiply(dataframe):
    no_zero_df                     = dataframe[dataframe.x_soll > 0] #selects the dataframe that has no zeros in x_soll
    time_of_step                   = no_zero_df.index[0] #returns the time at which the step occurs
    last_zero_df                   = dataframe[(dataframe.x_soll == 0) & (dataframe.index < time_of_step)].tail(1) #selects the df containing last zero value before the change in step 
    
    
    response_start_time            = pd.concat([last_zero_df, no_zero_df]).index[0] #returns the actual starting time of the dataframe
    data_with_initial_zero         = pd.concat([last_zero_df, no_zero_df]) #returns a dataframe that starts from a 0 value so that we get an actual'step'
    time_series_starting_from_zero = data_with_initial_zero.index-response_start_time #returns a new time series data whose time starts from zero
    data_for_modeling              = data_with_initial_zero.set_index(time_series_starting_from_zero) #changing the index of the dataframe with the new time series

    x_soll_steady_state            = no_zero_df.x_soll.head(1) #returns the steady state value along with the time index
    multiplication_factor          = 1 / (pd.Series(x_soll_steady_state).values[0]) #calculates the factor to be multiplied with each of the data
    input_array                    = (multiplication_factor * data_for_modeling.x_soll).tolist() #returns the 1D array  after multiplying the input with the factor in order to equalise it with 1 
    output_array                   = (multiplication_factor * data_for_modeling.x_ist).tolist() #returns the 1D array  after multiplying the output with the factor in order to equalise it with 1
    time_array                     = data_for_modeling.index.tolist() #extracting the time series index into a 1D array
    return input_array, output_array, time_array



#data1
df_ist_soll_1      = df_ist_soll_1[(df_ist_soll_1.x_ist > 0)]
xin_1, yout_1, t_1 = strip_multiply(df_ist_soll_1)
#plt.plot(t_1,yout_1)
#plt.plot(t_1,xin_1)


#data2
df_ist_soll_2      = df_ist_soll_2[(df_ist_soll_2.x_ist > 0)]
xin_2, yout_2, t_2 = strip_multiply(df_ist_soll_2)  
#plt.plot(t_2,yout_2)
#plt.plot(t_2,xin_2)


#data3-data with a special case trend
df_ist_soll_3          = df_ist_soll_3[(df_ist_soll_3.x_ist > 0)]
#the following code identifies the decreasing trend in x_ist values and removes those values from the dataframe
df_ist_soll_3['trend'] = np.sign(df_ist_soll_3['x_ist'].rolling(window = 5).mean().diff().fillna(0)).map({0:'FLAT', 1:'UP', -1:'DOWN'})
reversed_array = list(df_ist_soll_3.trend.values)[::-1]
counter = 0
for i in range(0, len(reversed_array)):
    if reversed_array[i] == 'DOWN':
        counter = counter + 1
    else:
        break
length             = len(df_ist_soll_3)
df_ist_soll_3      = df_ist_soll_3.head(length - counter)
xin_3, yout_3, t_3 = strip_multiply(df_ist_soll_3)  
#plt.plot(t_3,yout_3)
#plt.plot(t_3,xin_3)


#data4
df_ist_soll_4 = df_ist_soll_4[(df_ist_soll_4.x_ist > 0)]
xin_4, yout_4, t_4 = strip_multiply(df_ist_soll_4)  
#plt.plot(t_4,yout_4)
#plt.plot(t_4,xin_4)


yout = [yout_1, yout_2, yout_3, yout_4]
t = [t_1, t_2, t_3, t_4]

###### Section for getting the fitted model and aic values 

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R 
library(stats) #need this for using the ar() function
library(forecast) #need this for using the fitted.values() function

In [ ]:
#The function returns the aic,mse and fitted values for a given output data and a given order 
def order_ar(ar_order, output):
    %R -i ar_order,output #data is inputed as a list vector. In a list vector, each element in the list is an array. No space can be provided after comma. If provided, it results in 'NameError: name '' is not defined' error.
    %R output                 = unlist(output)  #converts the list vector back into a single array. Need this step in latest R version 3.4.2.
    %R ar_system              = ar(output, method = "ols", order.max = ar_order)
    %R fitted_ar_with_missing = fitted.values(ar_system) #the lower values in the output data contributes to NA/missing values in the fitted values. This corresponds to the delay in the output data.
    %R -o fitted_ar_with_missing
    
    fitted_ar_without_missing = np.nan_to_num(fitted_ar_with_missing) #the missing values becomes nan values in Python and they are converted to 0. It becomes easier in finding the delay for our model.
    mse_ar = mean_squared_error(output,fitted_ar_without_missing)
        
    %R -i mse_ar
    %R output_length = length(output) 
    %R aic_ar        = (output_length * log(mse_ar)) + (2 * ar_order) + (output_length * dim(matrix(output))[2] * (log(2 * pi) + 1)) #result obtained from https://rdrr.io/cran/sysid/src/R/estpoly.R
    %R -o aic_ar,mse_ar
    
    return aic_ar, mse_ar, fitted_ar_without_missing

###### MSE and AIC Dataframe for orders 1 to 10 along with fitted values

In [ ]:
#The function returns a dataframe that contains the aic,mse and order values from 1 to 10 along with the fitted values for each order 
def order_aic_mse_fit(yout):
    aic                  = []
    order                = []
    mse                  = []
    fit_array            = []
    aic_mse_fit_df       = [] 
    aic_mse_fit_df.append([])#2D array in which each element stores two floats(aic and mse) and an array(fit_array)
   
    for i in range(1,11):
        order.append(i)
        aic_mse_fit_df.append(order_ar(i,yout))
        aic.append(list(aic_mse_fit_df[i][0]))
        mse.append(aic_mse_fit_df[i][1])
        fit_array.append(aic_mse_fit_df[i][2])
    
    df = pd.DataFrame(np.column_stack([order, aic, mse]),columns=['order', 'aic', 'mse']) #all variables are passed into the dataframe as type float by default  
    return df, fit_array
    

df_1, fit_array_1 = order_aic_mse_fit(yout_1)
df_2, fit_array_2 = order_aic_mse_fit(yout_2)
df_3, fit_array_3 = order_aic_mse_fit(yout_3)
df_4, fit_array_4 = order_aic_mse_fit(yout_4)

###### Selecting the best order and the corresponding aic, mse and fitted values for each data based on low mse value

In [ ]:
order_1, mse_ar_1, aic_ar_1 = list(df_1[df_1.mse == df_1.mse.min()].values[0]) #list function lists the df row having the least mse into a 1D array 
fitted_1                    = fit_array_1[int(order_1) - 1]#int function converts the order back to the integer value. Order-1 gives the corresponding index value 

order_2, mse_ar_2, aic_ar_2 = list(df_2[df_2.mse == df_2.mse.min()].values[0])
fitted_2                    = fit_array_2[int(order_2) - 1]

order_3, mse_ar_3, aic_ar_3 = list(df_3[df_3.mse == df_3.mse.min()].values[0])
fitted_3                    = fit_array_3[int(order_3) - 1]

order_4, mse_ar_4, aic_ar_4 = list(df_4[df_4.mse == df_4.mse.min()].values[0])
fitted_4                    = fit_array_4[int(order_4) - 1]

#plt.plot(t_4,yout_4,label='original')
#plt.plot(t_4,fitted_4,label='model')
#plt.legend()

###### Smoothing all the model outputs

In [ ]:
def smooth(fitted_values, order):
    
    c = 0 #a counter that returns the number of zeros present in the dataframe
    for i in range(0,len(fitted_values)):
        if fitted_values[i] == 0.0:
            c = c + 1
        else:
            break

    fitted_without_zeros  = fitted_values[c:] #smoothing is done after stripping of the zeros
    multiplication_factor = len(fitted_without_zeros / 2) % 2 #calculates the factor to be multiplied in finding the window length
    window_length         = math.ceil((len(fitted_without_zeros) + 2 * multiplication_factor) / 2) #returns an odd integer, which is greater than half of the length of fitted data
    filter_output         = sig.savgol_filter(fitted_without_zeros, window_length, order) #the filter preserves the distributional features like relative maxima, minima and width of the time series
    smoothed_data         = np.append(fitted_values[0:c], filter_output)
    return smoothed_data      


smooth_11 = smooth(fitted_1, 1)
smooth_21 = smooth(fitted_2, 1)
smooth_31 = smooth(fitted_3, 1)
smooth_41 = smooth(fitted_4, 1)

smooth_12 = smooth(fitted_1, 2)
smooth_22 = smooth(fitted_2, 2)
smooth_32 = smooth(fitted_3, 2)
smooth_42 = smooth(fitted_4, 2)
#plt.plot(T_2,smooth_2,label='smoothed')
#plt.plot(T_2,fitted_2,label='fitted_values')
#plt.legend()

###### PT1 and PT2 Modeling on all data 

In [ ]:
def pt1(smooth, time):
    smoothed_df = pd.concat([pd.DataFrame(smooth, columns = ['smoothed']), pd.DataFrame(time, columns = ['time'])], axis = 1)
    steady_state = smooth[-1] #last element of the smoothed data is passed as steady state value
    standard_output = steady_state * (1 - np.exp(-1)) #case when t = T in the eqn. y(t) = steady_state * (1 - e ^ (−t / T)). 
    #delay = smoothed_df.time[smoothed_df.smoothed > 0.01].values[0] 
    delay = smoothed_df.time[smoothed_df.smoothed < 0.01].values[-1]#returns the time at which the step change occurs i.e a transition from 0 to some value. Values lesser than 0.01 are treatred as zero.
    time_constant = smoothed_df.time[smoothed_df.index == abs(smoothed_df.smoothed - standard_output).sort_values().index[0]].values[0] #derived from the equation of standard output
    tf_pt1 = con.matlab.tf(steady_state, [time_constant, 1])
    numerator, denominator = con.pade(delay, 1)
    delay_tf_pt1 = con.matlab.tf(numerator,denominator)
    yout_pt1, t_pt1 = con.matlab.step(tf_pt1 * delay_tf_pt1)
    return tf_pt1, delay_tf_pt1, yout_pt1, t_pt1


'''
The following function is based on the research aricle:
C. Huang and C. Chou, "Estimation of the underdamped second-order parameters from the system transient",
Industrial & Engineering Chemistry Research, vol. 33, no. 1, pp. 174-176, 1994. 
However, the delay calculation is based on visual inspection as stated in http://cse.lab.imtlucca.it/~bemporad/teaching/ac/pdf/AC2-08-System_Identification.pdf 
'''

def pt2(smooth, time):
    
    def fourpoint(z):
        f1_zeta = 0.451465 + (0.066696 * z) + (0.013639 * z ** 2)
        f3_zeta = 0.800879 + (0.194550 * z) + (0.101784 * z ** 2)
        f6_zeta = 1.202664 + (0.288331 * z) + (0.530572 * z ** 2)
        f9_zeta = 1.941112 - (1.237235 * z) + (3.182373 * z ** 2)
        return f1_zeta, f3_zeta, f6_zeta, f9_zeta
    
    def method1():
        beta = (t9 - t6) / (t3 - t1)
        zeta = -0.460805 + (0.976315 * beta) - (0.254517 * beta ** 2) + (0.028115 * beta ** 3)
        
        f1_zeta, f3_zeta, f6_zeta, f9_zeta = fourpoint(zeta)
        
        sum_ti        = t1 + t3 + t6 + t9
        sum_fi        = f1_zeta + f3_zeta + f6_zeta + f9_zeta
        sum_fi_sq     = (f1_zeta ** 2) + (f3_zeta ** 2) + (f6_zeta ** 2) + (f9_zeta ** 2)
        sum_ti_fi     = (t1 * f1_zeta) + (t3 * f3_zeta) + (t6 * f6_zeta) + (t9 * f9_zeta)
        time_constant = ((4 * sum_ti_fi) - (sum_fi * sum_ti)) / ((4 * sum_fi_sq) - sum_fi**2) #represented as T in the article 
        #delay        = ((sum_ti * sum_fi_sq) - (sum_fi * sum_ti_fi)) / ((4 * sum_fi_sq) - sum_fi**2) #based on article. Reprented as θ in the article
        delay         = smoothed_df.time[smoothed_df.smoothed < 0.01].values[-1] 
        return zeta, time_constant, delay
    
    def method2(): #the second method from the article is adopted because the zeta
        zeta = numpy.sqrt((numpy.log(overshoot) ** 2) / ((numpy.pi ** 2) + (numpy.log(overshoot) ** 2)))
        
        f1_zeta, f3_zeta, f6_zeta, f9_zeta = fourpoint(zeta)
        
        time_constant = (t9 - t1) / (f9_zeta - f1_zeta) 
        #delay        = t1 - time_constant*f1_zeta              #based on article.
        delay         = smoothed_df.time[smoothed_df.smoothed < 0.01].values[-1]  
        return zeta, time_constant, delay
    
    smoothed_df  = pd.concat([pd.DataFrame(smooth, columns = ['smoothed']), pd.DataFrame(time, columns = ['time'])], axis = 1)
    steady_state = smooth[-1]
    
    #ssn = steady state at n/10th instant of time   
    ss1 = steady_state * 0.1
    ss3 = steady_state * 0.3
    ss6 = steady_state * 0.6
    ss9 = steady_state * 0.9
    
    #tn = time at n/10th instant
    t1 = smoothed_df.time[smoothed_df.index == abs(smoothed_df.smoothed - ss1).sort_values().index[0]].values[0]
    t3 = smoothed_df.time[smoothed_df.index == abs(smoothed_df.smoothed - ss3).sort_values().index[0]].values[0]
    t6 = smoothed_df.time[smoothed_df.index == abs(smoothed_df.smoothed - ss6).sort_values().index[0]].values[0]
    t9 = smoothed_df.time[smoothed_df.index == abs(smoothed_df.smoothed - ss9).sort_values().index[0]].values[0]
       
    peak = smoothed_df.smoothed.max() #returns the highest output in the response
    overshoot = (peak - steady_state) /steady_state #represented as Mp in article
        
    if overshoot < 0.015:
        zeta, time_constant, delay = method1()
    else:
        zeta, time_constant, delay = method2()
   
    tf_pt2 = con.matlab.tf(steady_state, [time_constant ** 2, 2 * zeta * time_constant, 1])
    n_2, d_2 = con.pade(delay, 1)
    delay_tf_pt2 = con.matlab.tf(n_2, d_2)
    yout_pt2,t_pt2 = con.matlab.step(tf_pt2 * delay_tf_pt2)
    return tf_pt2, delay_tf_pt2, yout_pt2, t_pt2


#youto,to are the yout and t outputs from the pt1 and pt2 system
tf_11, delay_11, youto_11, to_11 = pt1(smooth_11, t_1)
tf_21, delay_21, youto_21, to_21 = pt1(smooth_21, t_2)
tf_31, delay_31, youto_31, to_31 = pt1(smooth_31, t_3)
tf_41, delay_41, youto_41, to_41 = pt1(smooth_41, t_4)
tf_12, delay_12, youto_12, to_12 = pt2(smooth_12, t_1)
tf_22, delay_22, youto_22, to_22 = pt2(smooth_22, t_2)
tf_32, delay_32, youto_32, to_32 = pt2(smooth_32, t_3)
tf_42, delay_42, youto_42, to_42 = pt2(smooth_42, t_4)

youto1 = [youto_11, youto_21, youto_31, youto_41]
to1    = [to_11, to_21, to_31, to_41]
youto2 = [youto_12, youto_22, youto_32, youto_42]
to2    = [to_12, to_22, to_32, to_42]

#plt.plot(to_2,youto_2)
#plt.plot(T_2,yout_2)

###### State space representation 

In [ ]:
#the function returns the steady state(ss) parameters for a given transfer function
def ss(TF):
    ss_parameters = con.matlab.tf2ss(TF)
    return ss_parameters

ss_11 = ss(tf_11 * delay_11)
ss_21 = ss(tf_21 * delay_21)
ss_31 = ss(tf_31 * delay_31)
ss_41 = ss(tf_41 * delay_41)
ss_12 = ss(tf_12 * delay_12)
ss_22 = ss(tf_22 * delay_22)
ss_32 = ss(tf_32 * delay_32)
ss_42 = ss(tf_42 * delay_42)
ss1 = [ss_11, ss_21, ss_31, ss_41]
ss2 = [ss_12, ss_22, ss_32, ss_42]

###### Mean square comparison of each data and its model

In [ ]:
#The function calculates mse between data and model. model_time samples are lesser than data_time. We take account of data_output corresponding to model output. So matching is necessary.   
def mse(data_time, data_output, model_output, model_time):
    data_output_df = pd.concat([pd.DataFrame(data_output, columns = ['data_output']), pd.DataFrame(data_time, columns = ['time'])], axis = 1)
    model_output_df = pd.concat([pd.DataFrame(model_output, columns = ['model_output']), pd.DataFrame(model_time, columns = ['time'])], axis = 1)
       
    #dt_match_mt = data_time matching with model_time
    dt_match_mt = [] #2d array because similar match values are passed as separate array inside the main array. Main array has a size as that of model_array.
    for i in range(0, len(model_time)):
        dt_match_mt.append([])
        for j in range(0, len(data_time)):
            if round(abs(model_time[i])) == round(abs(data_time[j])) or (round(model_time[i]) == round(data_time[j] + 0.5)) or (round(model_time[i] + 0.5) == round(data_time[j])): #allows matching of times with 0.5 accuracy
                dt_match_mt[i].append(data_time[j])
    #if the difference between model_time elements and matching data_time elements is minimum, then such values are passed into array 
    least_difference = [] 
    for i in range(0, len(model_time)):
        least_difference.append(min(abs(model_time[i] - dt_match_mt[i])))
    
    #data_time corresponding to model_time
    data_time_sliced = []
    for i in range(0, len(model_time)):
        for j in range(0, len(data_time)):
            if abs(model_time[i] - data_time[j]) == least_difference[i]:
                data_time_sliced.append(data_time[j])
    
    #data_output corresponding to data_time
    data_output_sliced = []
    for i in range(0, len(model_time)):
        data_output_sliced.append(list(data_output_df.data_output[data_output_df.time == data_time_sliced[i]])[0])
   
    mse = mean_squared_error(data_output_sliced, model_output)
    return mse

mse_11 = mse(t_1, yout_1, youto_11, to_11)
mse_21 = mse(t_2, yout_2, youto_21, to_21)
mse_31 = mse(t_3, yout_3, youto_31, to_31)
mse_41 = mse(t_4, yout_4, youto_41, to_41)
mse_12 = mse(t_1, yout_1, youto_12, to_12)
mse_22 = mse(t_2, yout_2, youto_22, to_22)
mse_32 = mse(t_3, yout_3, youto_32, to_32)
mse_42 = mse(t_4, yout_4, youto_42, to_42)
mse1 = [mse_11, mse_21, mse_31, mse_41]
mse2 = [mse_12, mse_22, mse_32, mse_42]

###### PT1 and PT2 Model response of all the dataset

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

def function_pt1(x):
    plt.plot(to1[x-1], youto1[x-1], label = 'pt1 model')
    plt.plot(t[x-1], yout[x-1], label = 'data')
    plt.legend()
    print('Mean square error is:')
    print(mse1[x-1])
    return ss1[x-1]

def function_pt2(x):
    plt.plot(to2[x-1], youto2[x-1], label = 'pt2 model')
    plt.plot(t[x-1], yout[x-1], label = 'data')
    plt.legend()
    print('Mean square error is:')
    print(mse2[x-1])
    return ss2[x-1]

#interact(function_pt1, x = widgets.IntSlider(min=1, max=4, step=1));
#interact(function_pt2, x = widgets.IntSlider(min=1, max=4, step=1));

def selection(s):
    if s == 'pt1':
        #interact(function_pt1,x = widgets.IntSlider(min=1,max=4,step=1));
        interact(function_pt1, x = widgets.RadioButtons(options = [1, 2, 3, 4], value = 1, description = 'Data:', disabled = False));
    else:
        #interact(function_pt2,x = widgets.IntSlider(min=1,max=4,step=1));
        interact(function_pt2, x = widgets.RadioButtons(options = [1, 2, 3, 4], value = 1, description = 'Data:', disabled = False));

interact(selection, s = widgets.RadioButtons(options = ['pt1', 'pt2'], value = 'pt1', description = 'System:', disabled = False));

###### Selecting the best model parameters among the dataset 

In [ ]:
#the below loop calculates the model with the lowest mse among both pt1 and pt2 models
for i in range(0, len(mse1)):
    for j in range(0, len(mse2)):
        if(mse1[i] == min(mse1)):
            i_low_mse = i
        if(mse2[j] == min(mse2)):
            j_low_mse = j
            
if mse2[j_low_mse] < mse1[i_low_mse]:
    plt.plot(to2[j_low_mse], youto2[j_low_mse], label = 'pt2 model')
    plt.plot(t[j_low_mse], yout[j_low_mse], label = 'data')
    plt.legend()
    print('The best model is a pt2 system and is obtained from dataset:', j_low_mse + 1) #index value + 1 gives the best dataset 
    print('Mean square error is:', mse2[j_low_mse])
    print('The state space parameters are:')
    print(ss2[j_low_mse])
else:
    plt.plot(to1[i_low_mse], youto1[i_low_mse], label = 'pt2 model')
    plt.plot(t[i_low_mse], yout[i_low_mse], label = 'data')
    plt.legend()
    print('The best model is a pt1 system and is obtained from dataset:', i_low_mse + 1)
    print('Mean square error is:', mse1[i_low_mse])
    print('The state space parameters are:')
    print(ss1[i_low_mse])

In [ ]:
end_time = list(time.localtime())[3:6]
h, m, s=np.subtract(end_time, start_time)
print('Total execution time is ', h ,' hours,', m,' minutes and ',s,' seconds')